In [ ]:
from time import sleep
import pandas as pd
import tkinter as tk
import requests
import numpy as np

import pyautogui

In [ ]:
def run_query(query):
    response = requests.post('https://api.tarkov.dev/graphql', json={'query': query})
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, query))

# lists all items
query_all_items_prices = """
{
    items {
        shortName
        id
        width
        height
        avg24hPrice
        changeLast48hPercent
        basePrice
        sellFor {
            price
            source
        }
    }
}
"""

def getAllItemsPrices():
    result = run_query(query_all_items_prices)
    #result = run_query(water_query)
    return result['data']['items']

In [ ]:
path_data = './data/'

In [ ]:
def items_dict_to_df(all_items):
    all_items_df = pd.DataFrame(columns=['name', 'id', 'width', 'height', 'icon_width', 'icon_height', 'features', 'fleaMarket', 'flea_avg48', 'flea_ch48percent', 'prapor', 'therapist', 'fence', 'skier', 'peacekeeper', 'mechanic', 'ragman', 'jaeger', 'basePrice'], index=range(len(all_items)))

    # iterate over all items
    for index in range(len(all_items)):
        all_items_df.iloc[index]['name'] = all_items[index].get('shortName')
        all_items_df.iloc[index]['id'] = all_items[index].get('id')
        all_items_df.iloc[index]['width'] = all_items[index].get('width')
        all_items_df.iloc[index]['height'] = all_items[index].get('height')
        all_items_df.iloc[index]['flea_avg48'] = all_items[index].get('avg24hPrice')
        all_items_df.iloc[index]['flea_ch48percent'] = all_items[index].get('changeLast48hPercent')
        all_items_df.iloc[index]['basePrice'] = all_items[index].get('basePrice')

        # iterate over all traders that can buy the item
        for offer in all_items[index].get('sellFor'):
            trader = offer.get('source')
            price  = offer.get('price')
            all_items_df.iloc[index][trader] = price
    
    return all_items_df


In [ ]:
# create the items dataframe
all_items_dict = getAllItemsPrices()

In [ ]:
all_items_df   = items_dict_to_df(all_items_dict)

In [ ]:
# save dataframe as csv
all_items_df.to_csv('../../' + path_data + 'all_items.csv', index=False)

In [ ]:
# load dataframe from csv
all_items_df = pd.read_csv('../../' + path_data + 'all_items.csv')

In [ ]:
a = all_items_df.loc[1000, 'prapor':'jaeger'].copy()
a['peacekeeper'] = 10
a

In [ ]:
item

In [ ]:
item = all_items_df.loc[100]
price_traders = item.loc['prapor':'jaeger']
price_max = np.nanmax(price_traders)
trader = np.nanargmax(price_traders)
trader

In [ ]:
# all_items_df

In [ ]:
# all_items_dict

## Pyautogui and keyboard

In [ ]:
import keyboard
from time import sleep
import pyautogui
from PIL import Image, ImageStat

In [ ]:
def get_image_around_mouse(position):
    slot_size = 64 * 1.2
    x = (int) (position[0] - 0.5*slot_size)
    y = (int) (position[1] - 0.5*slot_size)
    screenshot = pyautogui.screenshot(region=(x,y,slot_size,slot_size))
    screenshot = np.array(screenshot)
    return screenshot

def predict_item_under_mouse():
    mouse_pos = pyautogui.position()
    item = get_image_around_mouse(mouse_pos)
    prediction,distance = predict_icon(item)
    

In [ ]:
img = None
key = 'm'
while True:
    if keyboard.is_pressed(key):
        mouse_pos = pyautogui.position()
        print(f"Mouse Position: {mouse_pos}")
        img = get_image_around_mouse(mouse_pos)
        break
    if keyboard.is_pressed('escape'):
        break
        
Image.fromarray(img)

## JSON test

In [ ]:
import json

In [ ]:
file = open('../../config.jsonc')


In [ ]:
data = json.load(file)

In [ ]:
for v in data:
    print(v + ',', end='')

In [ ]:
data

In [ ]:
a = [1,2,3,4,1,6]
d = pd.DataFrame(a)

In [ ]:
b = d[0].nsmallest(3)

In [ ]:
max(b)